# Обработка нескольких соединений

* Как обработать несколько соединений одновременно?
* Что использовать, процессы или потоки?
* Рассмотрим примеры обработки сетевых запросов

## Потоки. Сервер

In [ ]:
import socket
import threading

def process_request(conn, addr):
    with conn:
        while True:
            # Метод recv() блокирует поток после получения данных от клиента в ожидание следующих данных:
            data = conn.recv(1024).decode('utf8')
            if not data.strip():
                break
            print(data, end='')


with socket.socket() as sock:
    sock.bind(('', 10001))
    sock.listen()
    
    while True:
        print('Ожидание нового соединения клиента на сокете')
        # Метод блокирующий поток, ожидающий соединений.
        conn, addr = sock.accept()
        print('Новое соединение клиента:', addr)
        # Создаем поток ОС
        th = threading.Thread(target=process_request, args=(conn, addr,))
        th.start() # Запускается поток обработки текущего соединения, а main поток идет дальше

        # В этом месте, основной поток продолжает исполнение и переходит на новую итерацию цикла while для ожидания
        # новых соединений, а запущенный поток обрабатывает существующее соединение.

## Потоки и процессы одновременно. Сервер

In [ ]:
import os
import socket
import threading
import multiprocessing


def process_request(conn, addr):
    with conn:
        while True:
            data = conn.recv(1024).decode('utf8')
            if not data.strip():
                break
            print(data, end='')

def worker(sock):
    while True:
        print('Ожидание нового соединения клиента на сокете. PID:', os.getpid())
        conn, addr = sock.accept() # Здесь происходит блокирование до появления новых данных в сокете.
        print('Новое соединение клиента:', addr)
        th = threading.Thread(target=process_request, args=(conn, addr,))
        th.start()
        # А в этом месте, основной поток переходит на новую итерацию цикла while для ожидания
        # новых соединений, а запущенный поток обрабатывает уже существующее соединение.


with socket.socket() as sock:
    sock.bind(('', 10001))
    sock.listen()

    # Пытаемся преодалеть ограничение GIL на потоки с помощью процессов.
    # Генерируется список из 3-х объектов Process которые будут принимать соединения на сокете
    workers_list = [multiprocessing.Process(target=worker, args=(sock,)) for _ in range(3)]

    for w in workers_list:
        w.start() # Запускаем процесс
        w.join()  # и запускаем ожидание главным процессом завершение дочерних процессов